# Filtering HPE

R code from Meckley2014
This code is for fixed tag and mobile tag test, for both the tag matching fish tag type.
Here we try it on the reference transmitters, which don't match the fish tag type (larger average delay).

In [ ]:
library(IRdisplay)

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
 </form>'
)

In [2]:
# load packages
require(PBSmapping)
library(ggplot2)
require(grid)
require(gridExtra)
require(sp)
library(rbokeh)


In [16]:
x=1:15
y=1:15
p <- ly_lines(x,y)
p

ERROR: Error in fig$x: $ operator is invalid for atomic vectors


This figure is empty...


In [10]:
p <- figure() %>%
  ly_points(Sepal.Length, Sepal.Width, data = iris,
    color = Species, glyph = Species,
    hover = list(Sepal.Length, Sepal.Width))
show(p)

## Step 1: Importing data

In [ ]:
# fixed tag data
estPos1 <- read.csv("~/Documents/DataHam/VPS/VPS-INBO-01-Results20160304/Positions/TRANSMITTER-S8-CALC-POSITIONS.csv")
# If available: drag tag data + gpsDrag: gps file from a trimble that was exactly above the drag
# tagDrag
# gpsDrag
# actual vps fish data
pos <- read.csv("~/Documents/DataHam/VPS/VPS-INBO-01-Results20160304/ALL-CALC-POSITIONS_onlyFishSheet.csv")
# receiver positions
rec2016 <- read.csv("~/Documents/DataHam/VPS/VPS-INBO-01-Results20160304/onlyStationLocations.csv")
# structure for viewing (csv file with contour of the sluices)
walls <- read.csv("~/Documents/Analysis/QGis visualisaties/walls-nodes.csv")
# polygon contouring the receiver array
POLYGON <- read.csv("~/Documents/Analysis/QGis visualisaties/recArray-nodes.csv")

## Step 2: Set configurations, fixed variables and limits to match study specifications

In [ ]:
# known Easting and Northing (UTM) for fixed tag location. If not known (like for R1): NA.
estPos1East <-NA
estPos1North <-NA
# average tag transmission rate of stationary and mobile tag (here only stationary)
# calculate the average delay with head(as.numeric(diff(estPos1$DATETIME)), -1), but as.numeric 
# is not working. For now just between min and max delay (540 and 660)
transRateStat <- 600
# transRateMobile
# Select HPE filter (first try, should be set after evaluating the array)
HPEfilter <- 8
# Map plotting limits: look at study site and identify min and max easting and northing
xAxisMax <-648200
xAxisMin <-647550
yAxisMax <-5662900
yAxisMin <-5662500
# Error Range of interest, just leave this for most applications
ErrorRange <- c(1:30)

# Accuracy Goals: These are determined by your reserach questions and analysis needs.
# CHECK THESE AGAIN!
# Outlier Maximum
OutlierGoal <- 15
# Accuracy Goal for all Points: 2DRMS <= accGoal (Trevor: 6m)
accGoal <- 1
# Goal for average accuracy if studying path lengths (1 order of magnitude less than max path length): mean error < 1.77
#avgAccGoal<- 1.77
# Setup for Evaluations
HPELow <- 2 #Lowest HPE to consider in plot.
HPEHigh<- 15 #Higest HPE to consider in plot.
#Length of Tag Drag in Seconds
#tagDragLength <- 4860 #recorded during the test
#UTC time zone difference to correct for variable times
#timeZoneDiff=4*60*60

## Step 3: Evaluate the array with fixed tag and tag drag data.

In [ ]:
# Convert Latitude longitude to UTM
tagll <- data.frame(X = estPos1$LON, Y = estPos1$LAT)
attr(tagll, "projection") <- "LL"
xy <- convUL(tagll) * 1000
# Rename Positions
tag1Easting<- xy[,1]
tag1Northing<- xy[,2]
estPos1$Easting<- tag1Easting
estPos1$Northing<- tag1Northing
# Median Point if a measured point is unavailable
estPos1East <- ifelse(is.na(estPos1East), median(estPos1$Easting),estPos1East)
estPos1North <- ifelse(is.na(estPos1North), median(estPos1$Northing),estPos1North)
# calculate distance between each point and the average
estPos1$error <- sqrt( (tag1Easting-(estPos1East))^2 + (tag1Northing-(estPos1North))^2)
# IF DRAG TEST AVAILABLE:
# What is the fix rate based on transmit rate and test length
# ...
# Process tag drag data
# ...
# Format receiver locations
# preparing receiver data for use in plots
tagll <- data.frame(X = rec2016$Longitude, Y=rec2016$Latitude)
attr(tagll, "projection") <- "LL"
xyrec <- convUL(tagll) * 1000
rec2016$Easting<- xyrec[,1]
rec2016$Northing<- xyrec[,2]
# More tagDrag stuff...


## Step 4: Relationship between HPE and measured accuracy (HPEm)

In [ ]:
#1. Bin by HPE
# Create increments to bin by
breaks <-seq(0, max(estPos1$HPE),by=0.1)
# Create a column with bins
estPos1$bin <- cut(estPos1$HPE,breaks)
#2. Calculate the average error (HPEm) for each 1 unit HPE bin
binMean <- tapply(estPos1$HPEm,estPos1$bin,mean)
# count the number of HPEs that fit into each bin value
binNum <- tapply(estPos1$HPEm,estPos1$bin,length)
binNum[is.na(binNum)] <- 0
# New dataframe for bin data
bin <- data.frame(binMean, binNum)
#3. Every HPE value in a given 1 m bin has a corresponding HPEm. Each of
    #these HPEm distances is composed of 2 elements: the error (difference
    #between the calculated and measured position) in the X direction (xe), and
    #the error in the Y direction (ye).
estPos1$xe <- sqrt((tag1Easting-(estPos1East))^2)
estPos1$ye <- sqrt((tag1Northing-(estPos1North))^2)
#4. For each bin, the standard deviations of Xe and Ye are calculated.
bin$xeSd <- tapply(estPos1$xe,estPos1$bin,sd)
bin$yeSd <- tapply(estPos1$ye,estPos1$bin,sd)
#5. To convert the 2-dimensional standard deviations calculated in #4 into a single measure, 
    #the 2DRMS error is calculated from the standard deviations of Xe and Ye
bin$RMS2d <- 2*sqrt((bin$xeSd)^2 + (bin$yeSd)^2)
#6. Now create a line plot and a dataframe just for the numbers we need, 
    #(ie when we have at least 10 tag transmissions and an HPE less than 21)
bin$avgHPE <- tapply(estPos1$HPE,estPos1$bin,mean)
smallBin <- bin[ which(bin$binNum > 10),]
smallBin <- smallBin[which(smallBin$avgHPE < 21 ),]
res3 <- lm(smallBin$RMS2d ~ smallBin$avgHPE)

In [ ]:
bin

Plot 1a: A close up of a fixed test to identify any irregular error bias (are there tails?).

In [ ]:
# PLOT THE POINT SPREAD
xyplot<- data.frame(tag1Easting, tag1Northing)
plot1a<- qplot(tag1Easting, tag1Northing , alpha= 1/100,data = xyplot, xlab='Easting (m)', 
               ylab= 'Northing (m) ') + theme_bw()
plot1a<- plot1a + geom_point(data=xyplot,aes(estPos1East,estPos1North), col="white", 
                             shape = 18, size= 4)
plot1a<- plot1a +theme(text = element_text(size=25), plot.title = element_text(vjust= 2), 
                       axis.text.y = element_text(angle=90, hjust= 0.5), 
                       axis.title.y = element_text(vjust= -0.02), 
                       axis.title.x = element_text(vjust= -0.02), 
                       plot.margin = unit(c(1,1.5,1,1.5), "cm"))
plot1a<- plot1a+theme(legend.position="none")#remove legend
#plot1a<- plot1a + coord_cartesian(xlim=c(647650,647750), ylim=c(5662750,5662850))
plot1a<- plot1a + coord_cartesian(xlim=c(xAxisMin,xAxisMax), ylim=c(yAxisMin,yAxisMax))
plot1a

In [ ]:
#Graph of HPE to error relationship
plot2a<- qplot(estPos1$HPE, estPos1$error , alpha= 1/100, data = xyplot, 
               xlab= 'HPE', ylab= 'Measured Error (m)')+ scale_y_continuous(limits = c(0, 21)) + 
                scale_x_continuous(limits = c(0, 21)) + theme_bw() +
                theme(plot.margin=unit(c(1,1.5,1,1.5), "cm"),panel.grid.major=element_blank(),
                panel.grid.minor=element_blank())+ theme(legend.position="none")
plot2a<- plot2a +theme(text = element_text(size=20),plot.title = element_text(vjust= 2), 
                       axis.title.y = element_text(vjust= -0.02), 
                       axis.title.x = element_text(vjust= -0.02))
plot2a <- plot2a + geom_point(data=smallBin, shape= 19, size= 5,col='black', 
                              bg= 'black',alpha= 1, aes(x = avgHPE, y = RMS2d)) + 
                                theme(legend.position = "none")
plot2a<- plot2a + geom_point(data=smallBin, shape= 19, size= 4.2,col='white', alpha= 1, 
                               aes(x = avgHPE, y = RMS2d)) + theme(legend.position = "none")
plot2a<- plot2a + geom_point(data=smallBin, shape= 4, size = 3,color='red', 
                             aes(x = avgHPE, y = RMS2d)) + theme(legend.position = "none")
plot2a<- plot2a +geom_abline(data=res3, col='white', aes(intercept=res3$coefficients[1], 
                                                         slope=res3$coefficients[2] ), size=1)
plot2a<- plot2a +geom_abline(data=res3, col='black', linetype='dashed', 
                             aes(intercept=res3$coefficients[1], slope=res3$coefficients[2]), 
                             size= 1)
# Add model equation and R2 to graph
# function to add linear model to graph
# Setup for a linear model to be added to a ggplot
lm_eqn = function(m) {
l <- list(a = format(coef(m)[1], digits = 2),
b = format(abs(coef(m)[2]), digits = 2),
r2 = format(summary(m)$r.squared, digits = 3));
if (coef(m)[2] >= 0) {
eq <- substitute(italic(y) == a + b %.% italic(x)*","~~italic(r)^2~"="~r2,l)
} else {
eq <- substitute(italic(y) == a - b %.% italic(x)*","~~italic(r)^2~"="~r2,l)
}
as.character(as.expression(eq));
}
# calls in linear model function for linear model res3
plot2a=plot2a + annotate("text",x = 8, y = 20,label = lm_eqn(res3), size= 6, parse=TRUE)

plot2a

Plot 2a: The HPE versus measured error to the median point (HPEm) is shown for each estimated position during the tests for a fixed test (black dots). The white circles with black outline and red x represent the two dimensional root mean square error of x and y components of error within an HPE bin of one.
95 percent of tag detections have an error less than this point within each bin. The line running between these points represents the two dimensional root means square error (2DRMS) and the equation and fit for this line are shown in the top left corner.

**COMMENT**
Normaal bin je slechts per HPE = 1, hier gebind per 0.2, anders slechts 2 punten. Niet echt mooi resultaat, vergeleken met vb...



## Step 5: Influence of HPE thresholds on fixed test data

In [ ]:
#What type of HPE cutoff should we use if we know what level of error we want
#!#! YOU MUST CHANGE accGoal to your desired error
#now evaluate how HPE cutoffs work if we want a specific error value
HPE<- c(1:HPEHigh)# we want a list of all possible HPE cutoffs
AllHPE <- data.frame(HPE)
AllHPE$ptsReject <- NA # number of points removed
AllHPE$ptsRejectP <- NA # percentage of all points
AllHPE$ptsRetain <- NA # number of poitns retained
AllHPE$ptsRetainP <- NA # percentage of all points
AllHPE$incorrectReject <- NA# good= error < accGoal and unacceptably erroneous= error > accGoal
AllHPE$incorrectRejectP <- NA
AllHPE$incorrectRetain<- NA
AllHPE$incorrectRetainP <- NA
AllHPE$incorrectRetainPvsRetain <-NA # incorretly retained/all retained *100
AllHPE$correctReject <- NA
AllHPE$correctRejectP <- NA
AllHPE$correctRetain <- NA
AllHPE$correctRetainP <- NA
AllHPE$goodDataLossP <- NA# incorrectReject/goodpts *100
AllHPE$badDataRetainP <- NA# incorrectRetain/AllHPE$ptsRetain * 100
AllHPE$sdDEV<-NA
AllHPE$avgErr<-NA
AllHPE$maxErr <- NA
# Number of all points that were unacceptably erroneous
badpts <-length(estPos1$error[estPos1$error > accGoal ])
# % of all points were unacceptably erroneous
badptsP <-(length(estPos1$error[estPos1$error > accGoal ])/(length(estPos1$error)))*100
#Number of all points that are good
goodpts <-length(estPos1$error[estPos1$error <= accGoal ])
#% of all points that are good
goodptsP <- (length(estPos1$error[estPos1$error <= accGoal ])/(length(estPos1$error)))*100

for( i in 1: length(AllHPE$HPE)){

    # which() gives al the indices where the logical is true => these are counted with length
    
    AllHPE$ptsReject[i] <- length(which(estPos1$HPE > AllHPE$HPE[i]))
    AllHPE$ptsRejectP[i] <- length(which(estPos1$HPE > AllHPE$HPE[i]))/(length(estPos1$error))*100 #% of all points dropped
    AllHPE$ptsRetain[i] <- length(which(estPos1$HPE < AllHPE$HPE[i]))
    AllHPE$ptsRetainP[i] <- length(which(estPos1$HPE < AllHPE$HPE[i]))/(length(estPos1$error))*100 #% of all points dropped
    
    #incorrect reject: error < than acceptable error, but HPE > filter (in AllHPE)
    AllHPE$incorrectReject[i] <- length(which((estPos1$HPE[estPos1$error <=accGoal] > AllHPE$HPE[i]) == TRUE))
    #divided by total with acceptable error
    AllHPE$incorrectRejectP[i] <-(AllHPE$incorrectReject[i]/length(estPos1$error[estPos1$error <= accGoal]))*100

    #incorrect retain
    AllHPE$incorrectRetain[i] <-length( which((estPos1$HPE[ estPos1$error >accGoal ] < AllHPE$HPE[i]) == TRUE))
    #divided by accurate points
    AllHPE$incorrectRetainP[i] <-(AllHPE$incorrectRetain[i]/length(estPos1$error[estPos1$error > accGoal ]))*100
    #divided by total retained by filter
    AllHPE$incorrectRetainPvsRetain[i] <-(AllHPE$incorrectRetain[i]/AllHPE$ptsRetain[i] )*100
    
    #Correctly Rejected
    AllHPE$correctReject[i] <-(length(which((estPos1$HPE[estPos1$error > accGoal] > AllHPE$HPE[i]) == TRUE)))
    #divided by sum of all positions 
    # QUESTION: here it is divided by all retained positions, not by all positions (like below)
    AllHPE$correctRejectP[i]<- (AllHPE$correctReject[i]/AllHPE$ptsRetain[i])*100
    
    #Correctly Retained
    AllHPE$correctRetain[i] <- length(which((estPos1$HPE[ estPos1$error < accGoal] < AllHPE$HPE[i]) == TRUE))
    #divided by sum of all positions
    AllHPE$correctRetainP[i]<-(AllHPE$correctRetain[i]/length(estPos1$error))*100
    
    AllHPE$goodDataLossP[i] <- (AllHPE$incorrectReject[i] / goodpts)*100
    AllHPE$badDataRetainP[i] <- (AllHPE$incorrectRetain[i]/AllHPE$ptsRetain[i])*100 
    AllHPE$avgErr[i] <- mean(estPos1$error[ estPos1$HPE < AllHPE$HPE[i]]) #mean of all error for each hpe
    AllHPE$sdDEV [i]<- sd(estPos1$error[ estPos1$HPE < AllHPE$HPE[i]]) #Standard deviation of all error for each HPE
    AllHPE$maxErr[i] <- max(estPos1$error[ estPos1$HPE < AllHPE$HPE[i]]) #max of all error for each hpe
    AllHPE$count15[i] <- length(which( (estPos1$HPE[ estPos1$error > 15] <AllHPE$HPE[i]) == TRUE ))
    }
    

In [ ]:
#Relationship between incorrectly rejected of acceptable data and incorrectly retained of all retained data.

start <- HPELow
end <- HPEHigh
AllHPEperf1 <- data.frame(HPE=AllHPE$HPE[start:end],perc=AllHPE$correctRejectP[start:end])
AllHPEperf1$var <-"Correct Rejection"
AllHPEperf2 <- data.frame(HPE=AllHPE$HPE[start:end],perc=AllHPE$incorrectRetainP[start:end])
AllHPEperf2$var <-"Incorrect Retainment"
AllHPEperf3 <- data.frame(HPE=AllHPE$HPE[start:end],perc=AllHPE$incorrectRejectP[start:end])
AllHPEperf3$var <-"Incorrect Rejection"
#Combine all seconts for graph
AllHPEperf <- rbind(AllHPEperf1,AllHPEperf3,AllHPEperf2)
#note legend is alphabetical but graph is by dataframe order

In [ ]:
plot3a <- qplot( AllHPE$incorrectRetainPvsRetain, AllHPE$incorrectRejectP,size=3, xlab= '% Incorrecly Retained 
(of all retained positions)',ylab= ' % Incorrectly Rejected \n (of all acceptable positions)' )+ylim(0,2) + xlim(3,4)+ theme_bw() + theme( 
    plot.margin = unit(c(1,1.5,1,1.5), "cm"),panel.grid.major=element_blank(), panel.grid.minor=element_blank())

plot3a<- plot3a +geom_line(data=AllHPE, aes(x=AllHPE$incorrectRetainPvsRetain,
                                            y=AllHPE$incorrectRejectP),color= 'black', size=0.25)
plot3a<- plot3a + theme_bw() + theme( text = element_text(size=25),
#plot.title = element_text(vjust= 2),
axis.text.y = element_text(angle=90, hjust= 0.4),
axis.title.y = element_text(vjust= .3),
axis.title.x = element_text(vjust= -0.02 ),
plot.margin = unit(c(1,1.5,1,1.5), "cm"),
panel.grid.major=element_blank(), panel.grid.minor=element_blank(),
legend.position = c(1.5, 0.8),
legend.background = element_rect(fill = "transparent",colour = NA),
legend.title=element_blank() )
plot3a <-plot3a+ geom_text(cex=6,aes(x=(AllHPE$incorrectRetainPvsRetain[3:15]),
y=(AllHPE$incorrectRejectP[3:15])+4 , label=AllHPE$HPE[3:15], group=NULL),)

plot3a

Plot 3: The figure informs how well HPE filters keep acceptably accurate data while removing unacceptable
data. The points are labeled by their associated HPE value.

In [ ]:
plot4a <- ggplot( data= AllHPE[start:end,],
aes(x = AllHPE$HPE[start:end])) +
geom_point(aes(y = AllHPE$avgErr[start:end],
shape= "Mean Error" ), size= 4, color= 'black') +
geom_point(aes(y = AllHPE$maxErr[start:end],
shape="Maximum Error" ), size= 4, color='black') +
geom_line(aes(y = OutlierGoal), size= 1, lty= 'dashed') +
#ggtitle(label6a)+
theme_bw() + theme( text = element_text(size=25),
plot.title = element_text(vjust= 2),
axis.text.y = element_text(angle=90, hjust= 0.5),
axis.title.y = element_text(vjust= -0.02 ),
axis.title.x = element_text(vjust= -0.02 ),
plot.margin = unit(c(1,1.5,1,1.5), "cm"),
panel.grid.major=element_blank(),
panel.grid.minor=element_blank(),
legend.position = c(0.35, 0.65),
legend.background = element_rect(
fill = "transparent",colour = NA),
legend.title=element_blank() ) +
xlab('HPE' ) +
coord_cartesian( ylim=c(0,45.25)) +
scale_y_continuous("Error (m)",breaks= round(seq(min(0),
max(35), by = 5),1)) +
scale_x_continuous("HPE",breaks=round(seq(min(AllHPE$HPE),
max(AllHPE$HPE), by = 1),1)) +
geom_text(cex=6,aes(x=AllHPE$HPE[3:15],y= (45),
label=AllHPE$count15[3:15], group=NULL),)

plot4a

Plot 4: The mean error, maximum measured error, and number of unacceptably high error values
are displayed at the top of the figure.

## Step 6: Influence of HPE cutoffs on tag drag data.

## Step 7: Influence of HPE cutoff on fish tracking data.

Identify internal and external array areas to compare performance.

In [ ]:
#Area of Receiver Coverage
tagll <- data.frame(X = POLYGON$Longitude, Y = POLYGON$Latitude)
attr(tagll, "projection") <- "LL"
xyrec <- convUL(tagll) * 1000
POLYGON$Easting<- xyrec[,1]
POLYGON$Northing<- xyrec[,2]

#convert fishpositions (pos) in easting, northing
tagll <- data.frame(X = pos$LON, Y = pos$LAT)
attr(tagll, "projection") <- "LL"
xyrec <- convUL(tagll) * 1000
pos$Easting<- xyrec[,1]
pos$Northing<- xyrec[,2]

#convert receiver locations (rec2016) in easting, northing
tagll <- data.frame(X = rec2016$Longitude, Y = rec2016$Latitude)
attr(tagll, "projection") <- "LL"
xyrec <- convUL(tagll) * 1000
rec2016$Easting<- xyrec[,1]
rec2016$Northing<- xyrec[,2]

#convert walls in easting, northing

tagll <- data.frame(X = walls$Longitude, Y = walls$Latitude)
attr(tagll, "projection") <- "LL"
xyrec <- convUL(tagll) * 1000
walls$Easting<- xyrec[,1]
walls$Northing<- xyrec[,2]


In [ ]:
#Evaluate Actual fish tracking data (HPE FILTER)
plot6a <- qplot(pos$Easting, pos$Northing, xlab= ' Easting (m)',
ylab= 'Northing (m)\n', size=I(1)) + theme_bw()
plot6a<- plot6a + coord_cartesian(xlim=c(xAxisMin,xAxisMax),
ylim=c(yAxisMin,yAxisMax))
plot6a<- plot6a +theme(text = element_text(size=25),
plot.title = element_text(vjust= 2),
axis.text.y = element_text(angle=90, hjust= 0.5),
axis.title.y = element_text(vjust= -0.02 ),
axis.title.x = element_text(vjust= -0.02 ),
plot.margin = unit(c(1,1.5,1,1.5), "cm") ) #make the axis
plot6a<- plot6a + geom_point(data=pos[pos$HPE < 3,], aes(x=Easting,
y=Northing),col="blue", shape = 20, size= I(1))  
plot6a<- plot6a + geom_point(data=pos[pos$HPE >= 3 &
pos$HPE < 5 ,],aes(x=Easting, y=Northing),col="#56B4E9",
shape = 20, size= I(1))
plot6a<- plot6a + geom_point(data=pos[pos$HPE >= 5 &
pos$HPE < 10 ,],aes(x=Easting, y=Northing),col="green",
shape = 20, size= I(1))
plot6a<- plot6a + geom_point(data=pos[pos$HPE >= 10 &
pos$HPE < 15 ,],aes(x=Easting, y=Northing),col="yellow",
shape = 20,size= I(1))
plot6a<- plot6a + geom_point(data=pos[pos$HPE >= 15 &
pos$HPE < 20 ,],aes(x=Easting, y=Northing),col="#FFCC33",
shape = 20,size= I(1))
plot6a<- plot6a + geom_point(data=pos[pos$HPE >= 20 ,],aes(x=Easting,
y=Northing),col="red", shape = 20, size= I(1))
# add receiver positions:
plot6a<- plot6a + geom_point(data=rec2016,aes(Easting, Northing),fill="black",
col="white", shape = 24, size= 2, alpha=1)
#+geom_text(data=rec2016,aes(Easting, Northing, label=rec2016$Station.Name),hjust=0, vjust=0, colour="red", size = 3)
# plot  delineation of study site (sluices)
plot6a<- plot6a + geom_path(data=walls,aes(Easting,Northing),col="black", size=1)

plot6a

Plot 6a: evaluate spatial trends in HPE for fish data

Identify if fish are inside or outside the array

In [ ]:
xydata<-data.frame(X=pos$Easting, Y=pos$Northing)
#create a polygon that covers the area of the data
fishPoly<- calcConvexHull(xydata, keepExtra=FALSE)
#Polygon(fishPoly)
#calcArea, calcCentroid are likley both useful
#Create a column in pos data.frame whether in or outside array
pos$array<- point.in.polygon(pos$Easting,pos$Northing, POLYGON$Easting,POLYGON$Northing, mode.checked= TRUE)
#plot of array polygon, with fish positions removed, 0 means not in array

# Filter data FOR FIRST SET OF PLOTS
cutoff <- HPEfilter
posDrop <- nrow(pos[(pos$HPE > cutoff),])
posDropArrayT <- ((pos$HPE[pos$array != 0]) > cutoff)
posDropArray <- length(subset(posDropArrayT, posDropArrayT == TRUE))
DropOutT <- ((pos$HPE[pos$array == 0]) > cutoff)
posDropOut <- length(subset( DropOutT, DropOutT == TRUE))
posTotal1<- nrow(pos)
posTotal2<- length(pos$array[pos$array == 0])
posTotal3<- length(pos$array[pos$array != 0])
percDropFil1 <- (posDrop/posTotal1)*100 # % DroppedTota;
percADropFil1 <- (posDropArray/posTotal2)*100 #% dropped in the array
percODropFil1 <- (posDropOut/posTotal3)*100 #% dropped outside of the array

#Now identify the silhoutte of the point cluster using calcConvexHull
xydata1<-data.frame(X=pos$EASTING[pos$HPE < cutoff ], Y=pos$NORTHING[pos$HPE <cutoff ])
#create a polygon that covers the area of the data
# fishPoly1<- calcConvexHull(xydata1, keepExtra=FALSE)
# head(xydata1)
# points(fishPoly1$X, fishPoly1$Y, col='red')
# #plot(pos$EASTING,pos$NORTHING)
# #calcArea, calcCentroid are likley both useful
# areaFil1 <- calcArea(fishPoly1) #calculates area of polygon


In [ ]:
#Plot 1- all fish positions
plot7a <- qplot(pos$Easting, pos$Northing, xlab= ' Easting (m)', ylab=
'Northing (m)',alpha=1/100, size=I(1)) + theme_bw()
plot7a <- plot7a + coord_cartesian(xlim=c(xAxisMin,xAxisMax), ylim=c(yAxisMin,yAxisMax))
plot7a <- plot7a + geom_point(data=rec2016,aes(Easting, Northing),col="red",
shape = 17, size= 2, alpha=1)
plot7a <- plot7a +theme(text = element_text(size=13),
plot.title = element_text(vjust= 2),
axis.text.y = element_text(angle=90, hjust= 0.5),
axis.title.y = element_text(vjust= -0.02 ),
axis.title.x = element_text(vjust= -0.02 ),
plot.margin = unit(c(1,1.5,1,1.5), "cm") ) #make the axis
plot7a <- plot7a + theme(legend.position="none") #remove legend

#plot 2- all positions with HPE < cutoff

plot7b <- qplot( pos$Easting[pos$HPE<= cutoff], pos$Northing[pos$HPE<= cutoff
], xlab= ' Easting (m)', ylab= 'Northing (m) ', alpha=1/100, size=I(1)) + theme_bw()
plot7b<- plot7b + coord_cartesian(xlim=c(xAxisMin,xAxisMax),
ylim=c(yAxisMin,yAxisMax))
plot7b<- plot7b + geom_point(data=rec2016,aes(Easting, Northing),col="red",
shape = 17, size= 2, alpha=1)
plot7b<- plot7b +theme(text = element_text(size=13),
plot.title = element_text(vjust= 2),
axis.text.y = element_text(angle=90, hjust= 0.5),
axis.title.y = element_text(vjust= -0.02 ),
axis.title.x = element_text(vjust= -0.02 ),
plot.margin = unit(c(1,1.5,1,1.5), "cm") ) #make the axis
plot7b<- plot7b + theme(legend.position="none") #remove legend

#plot 3- all positions with HPE > cutoff)
plot7c <- qplot( pos$Easting[pos$HPE> cutoff], pos$Northing[pos$HPE> cutoff],
xlab= ' Easting (m)', ylab= 'Northing (m) ',alpha=1/100, size=I(1)) +
theme_bw()
plot7c<- plot7c + coord_cartesian(xlim=c(xAxisMin,xAxisMax),
ylim=c(yAxisMin,yAxisMax))
plot7c<- plot7c + geom_point(data=rec2016,aes(Easting, Northing),col="red",shape = 17, size= 2, alpha=1)
plot7c<- plot7c +theme(text = element_text(size=13),
plot.title = element_text(vjust= 2),
axis.text.y = element_text(angle=90, hjust= 0.5),
axis.title.y = element_text(vjust= -0.02 ),
axis.title.x = element_text(vjust= -0.02 ),
plot.margin = unit(c(1,1.5,1,1.5), "cm") ) #make the axis
plot7c<- plot7c + theme(legend.position="none") #remove legend

par(mfrow=c(3,1))

plot7a
plot7b
plot7c

Plot7a-7c: Evaluate trends in data loss spatially for fish telemetry data for a selected HPE cutoff. 7a:
shows all fish data, 7b: shows remaining fish data following the use of a fixed HPE filter, and 7c: shows
removed fish data.